In [1]:
import sys
import os
import warnings
from pathlib import Path
import json
import math

import numpy as np
import matplotlib.pyplot as plt
import google.generativeai as genai
from dotenv import load_dotenv

if str(Path().resolve().parent) not in sys.path:
    sys.path.append(str(Path().resolve().parent))

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

warnings.filterwarnings("ignore", category=FutureWarning)

from src.config import PREPROCESSED_PAPER_DATASET_PATH, GENERATOR_ASSESSMENTS_PATH, GENERATOR_BEST_ASSESSMENTS_PATH
from src.text_extraction import *
from src.models_setup import embedding_model, gemini_2_flash
from src.prompts import prompt_rag, prompt_zero_cot, prompt_retry_with_memory_usage
from src.output_formats import BlogGeneration, BlogClassificationCoT
from src.helpers import get_examples, load_or_create_vector_store
from src.blog_generator import BlogGenerator
from src.long_term_memory import LongTermMemory

# Data Processing

In [2]:
# Import data
papers = pd.read_csv(PREPROCESSED_PAPER_DATASET_PATH)
papers = papers.reset_index(drop=True)

# Split dataset into validation and test set (first half for validation)
middle = papers.shape[0] // 2
Xval = papers.iloc[:middle].reset_index(drop=True)
Xtest = papers.iloc[middle:].reset_index(drop=True)

print(f"Size of validation set, X: {Xval.shape}")
print(f"Size of test set, X: {Xtest.shape}")

Size of validation set, X: (37, 4)
Size of test set, X: (37, 4)


# Generator manual test

In [3]:
vector_store = load_or_create_vector_store()

def find_most_similar_article(query_text):
    query_embedding = embedding_model.encode(query_text, clean_up_tokenization_spaces=True)
    results = vector_store.similarity_search_by_vector(query_embedding, k=2)

    if results:
        most_similar = results[1]
        return most_similar.page_content, most_similar.metadata

Vector store does not exist. Creating new one...
New vector store created successfully.


In [56]:
paper_text = Xval.loc[0, "paper_full_text"]
example_paper, example_blog_metadata = find_most_similar_article(paper_text)
example_blog = example_blog_metadata["blog_full_text"]

In [57]:
# RAG approach
llm_generator = gemini_2_flash.with_structured_output(BlogGeneration, include_raw=True)
generation_chain = prompt_rag | llm_generator
#print(prompt_rag.format(paper_text=paper_text,
#                        example_paper=example_paper,
#                        example_blog=example_blog))

In [58]:
generator_response = generation_chain.invoke({"paper_text": paper_text,
                                              "example_paper": example_paper,
                                              "example_blog": example_blog})

In [59]:
print(generator_response["parsed"].text)

# Minimum Hyperspherical Energy (MHE) in Neural Networks

## TL;DR

This blog post discusses a novel regularization technique called Minimum Hyperspherical Energy (MHE) for neural networks. MHE draws inspiration from the Thomson problem in physics to improve the generalization ability of neural networks by encouraging diversity among neurons.

## What's the Big Idea?

Neural networks are powerful, but their tendency to have redundant, highly correlated neurons can hurt performance. The goal is to regularize the network, avoiding this redundancy. The inspiration comes from the Thomson problem: how to distribute electrons on a sphere to minimize potential energy. MHE applies this concept to neurons, diversifying them to improve the network's generalization ability.

## Key Concepts

*   **Over-parametrization:** Neural networks often have more parameters than necessary, leading to redundancy.
*   **Generalization:** The ability of a neural network to perform well on unseen data.
*   **Re

In [60]:
examples = get_examples()
blog_text = generator_response["parsed"].text

llm_evaluator = gemini_2_flash.with_structured_output(BlogClassificationCoT, include_raw=True)
evaluation_chain = prompt_zero_cot | llm_evaluator
print(prompt_zero_cot.format(a="test for redundant additional parameters" ,blog_text=blog_text))

You are a very strict expert in evaluating written content, specializing in assessing how well blogs communicate scientific research to a broader audience.

Task:
Analyze the engagement level of the blog below based on the following criteria:
    - Readability
    - Structure
    - Informativeness
    - Attractiveness of the blog title
    - Clarity
    - Audience appeal
    - Potential for discussion
        
Clarifications:
    - Focus only on the textual content of the blog, disregarding any visual or interactive elements. This means that there is no need to add points related to the addition of illustrations or interactive elements to possible improvements.
    - Calmly lower your blog assessment according to the number of bugs.
    - Return ONLY a valid JSON object in plain text.

Expected Output Format:
    - Step by step, explain your analysis for each of the criteria listed above. Start by evaluating readability, then move on to structure, informativeness, and so on. Make sure 

In [61]:
evaluator_response = evaluation_chain.invoke({**examples, "blog_text": blog_text})

In [62]:
print(f"Overall assessment: {evaluator_response["parsed"].overall_assessment}")

Overall assessment: Very Good


In [63]:
improvements = evaluator_response["parsed"].improvements
print(f"Possible improvements:")
for i, improvement in enumerate(improvements):
    print(f"{i+1}. {improvement}")

Possible improvements:
1. Consider adding a section on the limitations of MHE and potential areas for future research.
2. Include a link to the official implementation, if available, could boost the blog attractiveness and potential for discussion.
3. Expand the explanation of the Thomson Problem to make it more accessible to readers without a physics background. This can be done by adding more intuitive examples or analogies, simplifying the explanation and adding a simple visual representation of the Thomson problem concept to improve understanding


In [64]:
possible_improvements = "\n".join([f"{i+1}. {improvement}" for i, improvement in enumerate(improvements)])
memory = LongTermMemory()
blog = generator_response["parsed"].text
similar_blog, metadata = memory.retrieve_memory(blog)

# Reflexion
generation_chain = prompt_retry_with_memory_usage | llm_generator

Loading long term memory module from: /home/kanstantsin-downar/PycharmProjects/bachalor-project/llm_blog_generator/data/long_term_memory
Long term memory module loaded successfully.
Found relevant memory.


In [65]:
generator_response = generation_chain.invoke({
    "generated_blog": blog,
    "possible_improvements": possible_improvements,
    "similar_blog": similar_blog,
    "similar_blog_score":
        metadata["overall_assessment"],
    "similar_blog_improvements":
        metadata["improvements"]
})

In [66]:
print(generator_response["parsed"].text)

# Minimum Hyperspherical Energy (MHE) in Neural Networks

## TL;DR

This blog post discusses a novel regularization technique called Minimum Hyperspherical Energy (MHE) for neural networks. MHE draws inspiration from the Thomson problem in physics to improve the generalization ability of neural networks by encouraging diversity among neurons.

## What's the Big Idea?

Neural networks are powerful, but their tendency to have redundant, highly correlated neurons can hurt performance. The goal is to regularize the network, avoiding this redundancy. The inspiration comes from the Thomson problem: how to distribute electrons on a sphere to minimize potential energy. MHE applies this concept to neurons, diversifying them to improve the network's generalization ability.

## Key Concepts

*   **Over-parametrization:** Neural networks often have more parameters than necessary, leading to redundancy.
*   **Generalization:** The ability of a neural network to perform well on unseen data.
*   **Re

In [67]:
blog_text = generator_response["parsed"].text
evaluator_response = evaluation_chain.invoke({**examples, "blog_text": blog_text})

In [68]:
print(f"Overall assessment: {evaluator_response["parsed"].overall_assessment}")

Overall assessment: Excellent


# End-to-end generation test

In [69]:
generator = BlogGenerator(use_memory=True, use_reflexion=True)

Loading vector store from: /home/kanstantsin-downar/PycharmProjects/bachalor-project/llm_blog_generator/data/vector_store
Vector store loaded successfully.
Loading long term memory module from: /home/kanstantsin-downar/PycharmProjects/bachalor-project/llm_blog_generator/data/long_term_memory
Long term memory module loaded successfully.
BlogGenerator initialized with vector store.


In [70]:
blog = generator.generate_blog(paper_text=Xval.loc[0, "paper_full_text"])

----------
Found most similar article.
Attempt number 1: Generating blog...
Using RAG prompt...
Checking request limits before invoking the model...
Invoking the model...
Model invoked successfully. Total requests today: 1, RPM: 1, TPM: 30323
Blog generated successfully.
Checking request limits before invoking the model...
Invoking the model...
Model invoked successfully. Total requests today: 2, RPM: 2, TPM: 32050
Blog evaluated successfully. Evaluation: Very Good.
No similar blogs found, new blog is added to memory.
Saving blog in file: /home/kanstantsin-downar/PycharmProjects/bachalor-project/llm_blog_generator/data/generation_results/blog
Long term memory module saved successfully.


In [71]:
print(blog)

# Regularizing Neural Networks with Hyperspherical Energy: A Deep Dive

Have you ever wondered why some neural networks generalize better than others? The secret might lie in how well the neurons are organized within the network. In a recent paper, researchers explored a fascinating connection between neural network regularization and a classic physics problem: the Thomson problem.

## The Thomson Problem: Evenly Distributing Electrons on a Sphere

The Thomson problem asks: how do you arrange N electrons on the surface of a sphere so that their electrostatic potential energy is minimized? In simpler terms, how do you spread out the electrons as evenly as possible, given that they all repel each other?

## Minimum Hyperspherical Energy (MHE): Applying the Thomson Problem to Neural Networks

The researchers drew inspiration from this problem to tackle a common issue in neural networks: redundancy. Over-parameterized networks often have highly correlated neurons, which can hurt their abil

# Experiments

In [72]:
def save_results(column, assessments, best_assessments):
    def create_new_json(error, res_type):
        print(f"{error}\n"
              f"Creating new json file for {res_type}...")
        json_file = {
            "RAG only": [],
            "Reflexion": [],
            "Reflexion + LTM": []
        }
        return json_file

    try:
        with open(GENERATOR_ASSESSMENTS_PATH, "r", encoding="utf-8") as f:
            results_generator_assessments = json.load(f)
    except Exception as err:
        results_generator_assessments = create_new_json(err, "Generator assessments")

    try:
        with open(GENERATOR_BEST_ASSESSMENTS_PATH, "r", encoding="utf-8") as f:
            results_generator_best_assessments = json.load(f)
    except Exception as err:
        results_generator_best_assessments = create_new_json(err, "Generator best assessments")

    results_generator_assessments[column] = assessments
    results_generator_best_assessments[column] = best_assessments

    with open(GENERATOR_ASSESSMENTS_PATH, "w", encoding="utf-8") as f:
        json.dump(results_generator_assessments, f)
    with open(GENERATOR_BEST_ASSESSMENTS_PATH, "w", encoding="utf-8") as f:
        json.dump(results_generator_best_assessments, f)

def run_experiment(history, best_history, dataset):
    redo = []
    for index, row in dataset.iterrows():
        assessment, best_assessment = generator.generate_blog(paper_text=row["paper_full_text"])
        if assessment and best_assessment:
            history[index] = assessment
            best_history[index] = best_assessment
        else:
            redo.append(index)
    return redo

def rerun_experiment_if_need(redo_list, history, best_history, dataset):
    while redo_list:
        redo_index = 0
        for index in redo_list:
            assessment, best_assessment = generator.generate_blog(paper_text=dataset.loc[index, "paper_full_text"])
            if assessment and best_assessment:
                history[index] = assessment
                best_history[index] = best_assessment
                del redo_list[redo_index]
                continue
            redo_index += 1

MAX_ATTEMPTS = 5
DATASET_SIZE = Xval.shape[0]
generator = BlogGenerator()

Loading vector store from: /home/kanstantsin-downar/PycharmProjects/bachalor-project/llm_blog_generator/data/vector_store
Vector store loaded successfully.
Loading long term memory module from: /home/kanstantsin-downar/PycharmProjects/bachalor-project/llm_blog_generator/data/long_term_memory
Long term memory module loaded successfully.
BlogGenerator initialized with vector store.


## RAG only approach

In [35]:
generator.max_attempts = MAX_ATTEMPTS
generator.experiment_mode = True
generator.set_usage_of_reflexion(False)
generator.set_usage_of_memory(False)

In [ ]:
assessment_history_only_RAG = [None] * DATASET_SIZE
best_assessment_history_only_RAG = [None] * DATASET_SIZE
redo_only_RAG = run_experiment(assessment_history_only_RAG, best_assessment_history_only_RAG, Xval)

In [ ]:
print(f"Experiment should be conduct again for blogs with ids: {redo_only_RAG}")

In [ ]:
rerun_experiment_if_need(redo_only_RAG, assessment_history_only_RAG, best_assessment_history_only_RAG, Xval)

In [15]:
save_results("RAG only", assessment_history_only_RAG, best_assessment_history_only_RAG)

## Reflexion

In [7]:
generator.set_usage_of_reflexion(True)

In [ ]:
assessment_history_ref = [None] * DATASET_SIZE
best_assessment_history_ref = [None] * DATASET_SIZE
redo_ref = run_experiment(assessment_history_ref, best_assessment_history_ref, Xval)

In [9]:
print(f"Experiment should be conduct again for blogs with ids: {redo_ref}")

Experiment should be conduct again for blogs with ids: [34, 45]


In [ ]:
rerun_experiment_if_need(redo_ref, assessment_history_ref, best_assessment_history_ref, Xval)

In [30]:
save_results("Reflexion", assessment_history_ref, best_assessment_history_ref)

## Reflexion + Long Term Memory

In [ ]:
generator.set_usage_of_memory(True)

In [ ]:
assessment_history_with_memory_usage = [None] * DATASET_SIZE
best_assessment_history_with_memory_usage = [None] * DATASET_SIZE
redo_with_memory_usage = run_experiment(assessment_history_with_memory_usage, best_assessment_history_with_memory_usage, Xtest)

In [ ]:
print(f"Experiment should be conduct again for blogs with ids: {redo_with_memory_usage}")

In [ ]:
rerun_experiment_if_need(redo_with_memory_usage, assessment_history_with_memory_usage, best_assessment_history_with_memory_usage, Xtest)

In [ ]:
save_results("Reflexion + LTM", assessment_history_with_memory_usage, best_assessment_history_with_memory_usage)

# Results

In [27]:
def plot(results_rag, results_ref, results_with_memory_usage, results_type):
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, 6), results_rag, marker="s", label="RAG only")
    plt.plot(range(1, 6), results_ref, marker="o", label="Reflexion")
    plt.plot(range(1, 6), results_with_memory_usage, marker="d", label="Reflexion + LTM")

    distance_from_edge = 0.2
    min_value = np.min([results_rag.min(), results_ref.min(), results_with_memory_usage.min()])
    plt.yticks([1, 2, 3, 4, 5], ["Bad", "Average", "Good", "Very Good", "Excellent"])
    plt.ylim(math.floor(min_value) - distance_from_edge, 5 + distance_from_edge)
    plt.xticks(range(1, 6))

    plt.xlabel("Number of Attempts")
    plt.ylabel(f"Average Engagement Level")
    plt.title(f"{results_type}")
    plt.legend()
    plt.grid(True)

    plt.show()

In [ ]:
try:
    with open(GENERATOR_ASSESSMENTS_PATH, "r", encoding="utf-8") as file:
        assessment_history = json.load(file)
except Exception as e:
    print(f"{e}\n"
          f"Results not founded.")

In [7]:
rag = np.array(assessment_history["RAG only"])
average_rag = np.mean(rag, axis=0)

reflexion = np.array(assessment_history["Reflexion"])
average_reflexion = np.mean(reflexion, axis=0)

ltm = np.array(assessment_history["Reflexion + LTM"])
average_ltm = np.mean(ltm, axis=0)

plot(average_rag, average_reflexion, average_ltm, "The Average Blog Engagement Level in This Attempt")

In [21]:
try:
    with open(GENERATOR_BEST_ASSESSMENTS_PATH, "r", encoding="utf-8") as file:
        best_assessment_history = json.load(file)
except Exception as e:
    print(f"{e}\n"
          f"Results not founded.")

In [ ]:
rag_best = np.array(best_assessment_history["RAG only"])
average_rag_best = np.mean(rag_best, axis=0)

reflexion_best = np.array(best_assessment_history["Reflexion"])
average_reflexion_best = np.mean(reflexion_best, axis=0)

ltm_best = np.array(best_assessment_history["Reflexion + LTM"])
average_ltm_best = np.mean(ltm_best, axis=0)

plot(average_rag_best, average_reflexion_best, average_ltm_best, "The Average Engagement Level of The Best Blog Generated Up To and Including This Attempt")